# Geospatial data analysis and visualization using xarray, matplotlib, cartopy, and MetPy
---

So far, we have used `matplotlib` to visualize data.  In this exercise, we will put it all together: 
- read files
- subset files using fancy indexing
- plot the results

We're going to use a new python package called `xarray`.  It is a high level interface for netCDF file reading and writing, that will make your life easier.  Let's go!  

In [ ]:
%pylab inline
import xarray as xr

Ahhhhhhhh....all better.  Let's get to work!

## Using xarray to read a netCDF file

netCDF4 is a common dataset for storing gridded binary data in atmospheric sciences.  It is a self-describing data format, meaning that it contains data and all of the coordinates necessary to use the data.  We will start with a simple example - maps of surface temperature anomalies from NASA GISS (from a server at NOAA ESRL).  This particular dataset is online and available for streaming through a service called OPENDaP - which means we don't even have to download the data.

It can't be much easier to read data than this...xarray handles a lot of the dirty work for you.  We can load both local files, as well as files on the internet like this OPENDaP file.  Xarray allows you to either give the local file path, or the web site!

Why xarray? It allows interfaces to self-describing data (such as netCDF) that is structured so you know exactly what you are getting, provides tools for easy analysis of the data.

<img src="https://github.com/pydata/xarray/raw/master/doc/_static/dataset-diagram.png">

In [ ]:
nc=xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/gistemp/combined/250km/air.2x2.250.mon.anom.comb.nc')
nc

In [ ]:
ncvar = nc['air']
ncvar


In [ ]:
air_tavg = ncvar.mean(dim='time')

In [ ]:
air_tavg.plot()

This data is a gridded time series at 2 degree resolution of monthly surface temperature anomalies starting in 1880.  Let's average over all space dimensions (lat - axis 1, and lon - axis 2).  We can use `np.mean` and its `axis` keyword (very handy) for this purpose.

This will yield a time series of globally averaged temperature!

In [ ]:
plt.figure()
plt.plot(ncvar.time,np.mean(ncvar,axis=(1,2)))
plt.xlabel('Year')
plt.ylabel('Temperature anomaly deg C')
plt.title("It's getting hot up in here!")
plt.show()

In [ ]:
ncvar.mean(dim=['lat','lon']).plot()
plt.xlabel('Years since some time')

We can use the select tool to get a subset in a box (find closest index values of lon and lat) so that we can subset the data and grab the closest point to Champaign-Urbana.  We can give it a list of points.  Here we will give it one.

In [ ]:
nc_cmi=nc.sel(lon=-88.9+360.,lat=40., method='nearest')
nc_cmi

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

var='air'

plt.plot(nc_cmi.time,nc_cmi[var])
plt.xlabel('Year')
plt.ylabel('Temperature anomaly deg C')
plt.title("It's not getting as hot in Champaign")

In [ ]:
nc['air'].sel(lon=-88.9+360.,lat=40., method='nearest').plot()

## Saving to file - easy as np.pi()
Want to save the file as a netCDF file?  No problem!

In [ ]:
nc_cmi.to_netcdf('nc_cmi.nc')

## Calculating time averages

Let's say we want to average the monthly time series data into annually averaged data.

There are a number of ways to do this.  `xarray` offers time sampling capabilities, similar to `pandas`. 

In [ ]:
nc_cmi

`xarray` and `pandas` share the same interface to resample and group time series conveniently.  The documentation is available at: http://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations.  The codes for resampling are the same as `pandas`.  See http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases


In [ ]:
nc_cmi3=nc_cmi.resample(time='AS').mean()
nc_cmi3

How easy is that?  np.pi()?  You can also resample with other time frequencies, or in space, or change how you do the calculation (i.e., calculate the median instead of the mean).

Now save to a file:

In [ ]:
site='Champaign_annualavgs'

nc_cmi3.to_netcdf(site+'_data2.nc')

Make a (nice) plot!

In [ ]:
plt.figure(figsize=(11,8.5)) #create a new figure

plt.plot(nc_cmi['time'],np.squeeze(nc_cmi['air']),'b',alpha=0.5)
plt.plot(nc_cmi3['time'],np.squeeze(nc_cmi3['air']),'r',linewidth=2.0)
plt.legend(['Monthly averages','Annual averages'])
plt.xlabel('Year')
plt.ylabel('Temperature Anomaly (degrees C)')
plt.title('GISTEMP Temperature Anomalies near Champaign, IL')
plt.show()

## Collapsing dimensions to make a Hovmoller Diagram

It's also easy to slice and dice the data to make a Hovmoller Diagram (time,longitude plot averaging over latitude)

In [ ]:
temp1= nc['air'].resample(time='AS').mean()
temp1.mean(dim='lat').plot(vmin=-4,vmax=4,cmap='seismic')

Mapping with `cartopy`
---

In [ ]:
%pylab inline

import xarray as xr
import pandas as pd

## Introduction to mapping with `cartopy`.

Python's `cartopy` is a package that is designed to enable `matplotlib` handle geographic calculations and visualize maps with geographic boundaries.  We can then use `matplotlib` to make maps, and off we go!

Cartopy has exposed an interface to enable easy map creation using matplotlib. Creating a basic map is as simple as telling matplotlib to use a specific map projection, and then adding some coastlines to the axes:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)

plt.show()

A list of the available projections to be used with `matplotlib` can be found on the Cartopy projection list page.

The line `plt.axes(projection=ccrs.PlateCarree())` sets up a `GeoAxes` instance which exposes a variety of other map related methods, in the case of the previous example, we used the `coastlines()` method to add coastlines to the map.

Lets create another map in a different projection, and make use of the `stock_img()` method to add an underlay image to the map:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

plt.figure(figsize=(11,11))
ax = plt.axes(projection=ccrs.Mollweide())
ax.coastlines()

ax.stock_img()

plt.show()

## Adding data to the map
Once you have the map just the way you want it, data can be added to it in exactly the same way as with normal matplotlib axes. By default, the coordinate system of any data added to a GeoAxes is the same as the coordinate system of the GeoAxes itself, to control which coordinate system that the given data is in, you can add the `transform` keyword with an appropriate `cartopy.crs.CRS` instance:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.Orthographic(central_longitude=20,central_latitude=40))
ax.coastlines()

ny_lon, ny_lat = -75, 43
delhi_lon, delhi_lat = 77.23, 28.61

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='blue', linewidth=2, marker='o',
         transform=ccrs.Geodetic(),
         )

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='gray', linestyle='--',
         transform=ccrs.PlateCarree(),
         )

plt.text(ny_lon - 3, ny_lat - 12, 'New York',
         horizontalalignment='right',
         transform=ccrs.Geodetic())

plt.text(delhi_lon + 3, delhi_lat - 12, 'Delhi',
         horizontalalignment='left',
         transform=ccrs.Geodetic())

plt.show()

# More advanced mapping with `cartopy` and `matplotlib`

From the outset, `cartopy`’s purpose has been to simplify and improve the quality of mapping visualisations available for scientific data. Thanks to the simplicity of the cartopy interface, in many cases the hardest part of producing such visualisations is getting hold of the data in the first place. 

In [ ]:
import os
import matplotlib.pyplot as plt

import numpy as np

from cartopy import config
import cartopy.crs as ccrs


# get the path of the file. 
sst_data = xr.open_dataset('https://www.ncei.noaa.gov/thredds/dodsC/ncFC/fc-oisst-daily-avhrr-only-dly/OISST_Daily_AVHRR-only_Feature_Collection_best.ncd').sel(time='2020-08-01').isel(zlev=0)

sst_data

In [ ]:
plt.figure(figsize=(10,5))

ax = plt.axes(projection=ccrs.PlateCarree())

plt.pcolormesh(sst_data['lon'], sst_data['lat'], sst_data['sst'].isel(time=0),
             transform=ccrs.PlateCarree(), cmap='viridis')

plt.title(sst_data.title+'\n'+str(sst_data['time'].values))
ax.coastlines()

plt.colorbar()
plt.show()

## Vector plotting

Cartopy comes with powerful vector field plotting functionality. There are 3 distinct options for visualising vector fields: quivers, barbs, and streamplots, each with their own benefits for displaying certain vector field forms.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import cartopy
import cartopy.crs as ccrs


def sample_data(shape=(20, 30)):
    """
    Returns ``(x, y, u, v, crs)`` of some vector data
    computed mathematically. The returned crs will be a rotated
    pole CRS, meaning that the vectors will be unevenly spaced in
    regular PlateCarree space.

    """
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)

    x = np.linspace(311.9, 391.1, shape[1])
    y = np.linspace(-23.6, 24.8, shape[0])

    x2d, y2d = np.meshgrid(x, y)
    u = 10 * (2 * np.cos(2 * np.deg2rad(x2d) + 3 * np.deg2rad(y2d + 30)) ** 2)
    v = 20 * np.cos(6 * np.deg2rad(x2d))

    return x, y, u, v, crs

plt.figure(figsize=(10,10))

ax = plt.axes(projection=ccrs.Orthographic(-10, 45))

ax.add_feature(cartopy.feature.OCEAN, zorder=0)
ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')

ax.set_global()
ax.gridlines()

x, y, u, v, vector_crs = sample_data()
ax.streamplot(x, y, u, v, transform=vector_crs)

plt.show()

## Putting it back together

Now, let's work with our global temperature anomaly dataset.  We will use xarray to select some times, and visualize the anomalies on a two panel plot.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.ticker as mticker

In [ ]:
nc=xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/gistemp/combined/250km/air.2x2.250.mon.anom.comb.nc')
nc

In [ ]:
#Select times, make lats and lons 2D so basemap knows where to plot each point

jan1976 = nc.sel(time=pd.datetime(1976,1,1))
jan2018 = nc.sel(time=pd.datetime(2018,1,1))

Here, let's plot the anomalies from these two time periods on a rectangular map projection called "PlateCarree" in a two panel plot.

In [ ]:
jan2018['air'].plot.pcolormesh()

In [ ]:
# select data we want
data = jan1976['air']
data2 = jan2018['air']

# create figure, axes instances.
fig = plt.figure(figsize=(12, 3))

ax = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree(central_longitude=180.))
data.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax.set_global(); 
ax.coastlines();
#set up the gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')
#set where the gridlines go
gl.xlocator = mticker.FixedLocator(np.arange(-180,181,15))
gl.ylocator = mticker.FixedLocator(np.arange(-90,91,15))

ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
data2.plot.contourf(ax=ax2, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax2.set_global(); 
ax2.coastlines();

gl2 = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')

gl2.ylocator = mticker.FixedLocator(np.arange(-90,91,15))
gl2.xlocator = mticker.FixedLocator(np.arange(-180,181,15))

plt.show()

Let's zoom in to the Eastern US - and add some fancy stuff like country and state borders, and lakes.

In [ ]:
import cartopy.feature as cfeature

# select data we want
data = jan1976['air']
data2 = jan2018['air']

# create figure, axes instances.
fig = plt.figure(figsize=(12, 6))

cartopy_proj = projection=ccrs.Mercator()

ax = plt.subplot(1, 2, 1, projection=cartopy_proj)
data.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax.set_extent([-100,-80,30.,50.], crs=ccrs.PlateCarree()) 
ax.coastlines()
# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')
ax.add_feature(states_provinces, edgecolor='gray')
ax.add_feature(cfeature.LAKES, edgecolor='gray')
ax.add_feature(cfeature.BORDERS, edgecolor='gray')
#set up the gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=2, color='gray', alpha=0.5, linestyle='-')
#set where the gridlines go
gl.xlocator = mticker.FixedLocator(np.arange(-180,181,5))
gl.ylocator = mticker.FixedLocator(np.arange(-90,90,5))

ax2 = plt.subplot(1, 2, 2, projection=cartopy_proj)
data2.plot.contourf(ax=ax2, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax2.set_extent([-100,-80,30.,50.], crs=ccrs.PlateCarree()) 
ax2.coastlines()
# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')
ax2.add_feature(states_provinces, edgecolor='gray')
ax2.add_feature(cfeature.LAKES, edgecolor='gray')
ax2.add_feature(cfeature.BORDERS, edgecolor='gray')
gl2 = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=2, color='gray', alpha=0.5, linestyle='-')

gl2.ylocator = mticker.FixedLocator(np.arange(-90,90,5))
gl2.xlocator = mticker.FixedLocator(np.arange(-180,181,5))

plt.show()